In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import axes
import pickle
from datetime import datetime
from scipy import log,exp,sqrt,stats
print('Packages')

Packages


In [2]:
# Importing pickle file
data = np.load('/Users/louisribieras/Desktop/IXPERIENCE/my_invictus/data/latest_coin_data.pkl')
# Drop Null Values
data = data.dropna()

print('Loading of Data')
print('shape of data: ', data.shape)

Loading of Data
shape of data:  (133, 916)


In [3]:
print('Making Volumes dataframe (daily): volumes')

# Initialize a close_price list
volumes_list = list()
# Making a list of all close_prices
for date in data.index:
    for column in data.columns:
        volumes_list.append(data[column][date]['volume'])

# Splitting list into 133 lists (one per coin)
volumes_list = [volumes_list[i:i+916] for i  in range(0, len(volumes_list), 916)]   

volumes = pd.DataFrame(data = volumes_list, index = data.index, columns = data.columns)
# Reordering column index
volumes = volumes[volumes.columns[::-1]]

# Index to Datetime index
volumes.columns = pd.to_datetime(volumes.columns)

# Deleting first day
del volumes['2016-01-03 00:00:00']

# original date index
data_index = volumes.columns

print('Shape of Volumes df: ', volumes.shape)
volumes.head(3)

Making Volumes dataframe (daily): volumes
Shape of Volumes df:  (133, 915)


,2016-01-01 00:00:00,2016-01-02 00:00:00,2016-01-04 00:00:00,2016-01-05 00:00:00,2016-01-06 00:00:00,2016-01-07 00:00:00,2016-01-08 00:00:00,2016-01-09 00:00:00,2016-01-10 00:00:00,2016-01-11 00:00:00,...,2018-06-25 00:00:00,2018-06-26 00:00:00,2018-06-27 00:00:00,2018-06-28 00:00:00,2018-06-29 00:00:00,2018-06-30 00:00:00,2018-07-01 00:00:00,2018-07-02 00:00:00,2018-07-03 00:00:00,2018-07-04 00:00:00
1337coin,24.0,12.0,3.0,13.0,2.0,27.0,20.0,16.0,121.0,18.0,...,219.0,2487.0,3683.0,764.0,4477.0,508.0,769.0,10084.0,1021.0,2343.0
advanced-internet-blocks,80.0,60.0,4.0,65.0,89.0,97.0,188.0,91.0,34.0,92.0,...,1.0,47.0,3.0,4.0,39.0,3.0,1.0,15.0,75.0,44.0
adzcoin,1486.0,1269.0,1134.0,1151.0,1021.0,2124.0,842.0,749.0,721.0,356.0,...,1280.0,1804.0,312.0,1169.0,2599.0,1172.0,2908.0,1480.0,925.0,727.0


In [4]:
print('Making Volumes dataframe (weekly): volumes_w')

# Initialize a close_price list
volumes_list2 = list()
# Making a list of all close_prices
for date in data.index:
    for column in data.columns:
        volumes_list2.append(data[column][date]['volume'])

# Splitting list into 133 lists (one per coin)
volumes_list2 = [volumes_list2[i:i+916] for i  in range(0, len(volumes_list2), 916)]   

volumes_w = pd.DataFrame(data = volumes_list2, index = data.index, columns = data.columns)

# Reordering column index
volumes_w = volumes_w[volumes_w.columns[::-1]]

# Index to Datetime index
volumes_w.columns = pd.to_datetime(volumes_w.columns)

# Weekly Median volumes
volumes_w = volumes_w.resample('W', axis = 1).median()

# Deleting first day
del volumes_w['2016-01-03 00:00:00']

# original date index (weekly)
data_index_w = volumes_w.columns

print('Shape of Weekly Volumes df: ', volumes_w.shape)
volumes_w.head(5)

Making Volumes dataframe (weekly): volumes_w
Shape of Weekly Volumes df:  (133, 131)


,2016-01-10 00:00:00,2016-01-17 00:00:00,2016-01-24 00:00:00,2016-01-31 00:00:00,2016-02-07 00:00:00,2016-02-14 00:00:00,2016-02-21 00:00:00,2016-02-28 00:00:00,2016-03-06 00:00:00,2016-03-13 00:00:00,...,2018-05-06 00:00:00,2018-05-13 00:00:00,2018-05-20 00:00:00,2018-05-27 00:00:00,2018-06-03 00:00:00,2018-06-10 00:00:00,2018-06-17 00:00:00,2018-06-24 00:00:00,2018-07-01 00:00:00,2018-07-08 00:00:00
1337coin,16.0,14.0,36.0,80.0,42.0,57.0,149.0,136.0,242.0,241.0,...,3693.0,1920.0,6996.0,2476.0,1274.0,1941.0,1257.0,1778.0,769.0,2343.0
advanced-internet-blocks,89.0,88.0,84.0,9.0,4.0,1.0,2.0,2.0,1.0,757.0,...,34.0,364.0,9.0,4.0,4.0,10.0,16.0,9.0,3.0,44.0
adzcoin,1021.0,917.0,319.0,843.0,325.0,268.0,418.0,209.0,267.0,1254.0,...,4292.0,2605.0,1886.0,5867.0,5149.0,7427.0,1724.0,2537.0,1280.0,925.0
aeon,317.0,137.0,55.0,79.0,269.0,460.0,464.0,443.0,456.0,127.0,...,81897.0,89235.0,70397.0,58968.0,36466.0,29367.0,42113.0,18753.0,13597.0,15181.0
anarchistsprime,2.0,1.0,2.0,8.0,3.0,5.0,6.0,101.0,42.0,86.0,...,74.0,73.0,4.0,50.0,33.0,16.0,45.0,6.0,23.0,184.0


In [5]:
print('Close Price dataframe: close_df')
# Initialize a close_price list
close_list = list()
# Making a list of all close_prices
for date in data.index:
    for column in data.columns:
        close_list.append(data[column][date]['close_price'])

# Splitting list into 133 lists (one per coin)
close_list2 = [close_list[i:i+916] for i  in range(0, len(close_list), 916)]

# Making a Dataframe
close_df = pd.DataFrame(data = close_list2, index = data.index, columns = data.columns)

# Reordering columns
close_df = close_df[close_df.columns[::-1]]

# Index to Datetime index
close_df.columns = pd.to_datetime(close_df.columns)

# Deleting first day
del close_df['2016-01-03 00:00:00']

close_df.head(3)

Close Price dataframe: close_df


,2016-01-01 00:00:00,2016-01-02 00:00:00,2016-01-04 00:00:00,2016-01-05 00:00:00,2016-01-06 00:00:00,2016-01-07 00:00:00,2016-01-08 00:00:00,2016-01-09 00:00:00,2016-01-10 00:00:00,2016-01-11 00:00:00,...,2018-06-25 00:00:00,2018-06-26 00:00:00,2018-06-27 00:00:00,2018-06-28 00:00:00,2018-06-29 00:00:00,2018-06-30 00:00:00,2018-07-01 00:00:00,2018-07-02 00:00:00,2018-07-03 00:00:00,2018-07-04 00:00:00
1337coin,0.000013,0.000013,0.000009,0.000013,0.000010,0.000017,0.000019,0.000022,0.000019,0.000018,...,0.000063,0.000087,0.000111,0.000058,0.000062,0.000061,0.000062,0.000066,0.000063,0.000065
advanced-internet-blocks,0.001429,0.000733,0.003461,0.003447,0.003425,0.003646,0.003581,0.003536,0.008036,0.006718,...,0.005340,0.002552,0.003626,0.003333,0.002476,0.002682,0.002665,0.005060,0.005007,0.005061
adzcoin,0.035203,0.033206,0.024340,0.032073,0.027896,0.017795,0.017245,0.017908,0.017920,0.019771,...,0.021635,0.019316,0.019415,0.019147,0.018698,0.020574,0.016432,0.017592,0.018273,0.018122


In [6]:
print('Calculating pct_change in close prices per week: close_pct')
weekly_close_df2 = close_df.copy().T
close_pct = weekly_close_df2.pct_change(1).T
close_pct.head(5)

Calculating pct_change in close prices per week: close_pct


,2016-01-01 00:00:00,2016-01-02 00:00:00,2016-01-04 00:00:00,2016-01-05 00:00:00,2016-01-06 00:00:00,2016-01-07 00:00:00,2016-01-08 00:00:00,2016-01-09 00:00:00,2016-01-10 00:00:00,2016-01-11 00:00:00,...,2018-06-25 00:00:00,2018-06-26 00:00:00,2018-06-27 00:00:00,2018-06-28 00:00:00,2018-06-29 00:00:00,2018-06-30 00:00:00,2018-07-01 00:00:00,2018-07-02 00:00:00,2018-07-03 00:00:00,2018-07-04 00:00:00
1337coin,NaN,0.000000,-0.307692,0.444444,-0.230769,0.700000,0.117647,0.157895,-0.136364,-0.052632,...,-0.015625,0.380952,0.275862,-0.477477,0.068966,-0.016129,0.016393,0.064516,-0.045455,0.031746
advanced-internet-blocks,NaN,-0.487054,3.721692,-0.004045,-0.006382,0.064526,-0.017828,-0.012566,1.272624,-0.164012,...,0.155344,-0.522097,0.420846,-0.080805,-0.257126,0.083199,-0.006339,0.898687,-0.010474,0.010785
adzcoin,NaN,-0.056728,-0.267000,0.317707,-0.130234,-0.362095,-0.030908,0.038446,0.000670,0.103292,...,0.001806,-0.107187,0.005125,-0.013804,-0.023450,0.100332,-0.201322,0.070594,0.038711,-0.008264
aeon,NaN,-0.075457,-0.094809,0.066587,-0.038981,0.035434,-0.081651,-0.085560,-0.347006,0.260572,...,0.017094,-0.016807,0.017094,-0.050420,0.026549,0.060345,0.016260,0.064000,-0.037594,0.023438
anarchistsprime,NaN,0.037129,-0.038186,-0.057072,-0.050000,0.058172,0.020942,-0.046154,-0.032258,0.108333,...,-0.007099,-0.275111,0.007985,-0.043336,-0.034096,0.030257,0.366618,0.041189,-0.015824,0.010835


In [7]:
# Reverting Index
close_df.columns = data_index
volumes.columns = data_index
volumes_w.columns = data_index_w
print('Reverting Index')

Reverting Index


In [10]:
print('Making weekly volume DataFrames')
# Making weekly volume DataFrames
wv = {}
for week in volumes_w.columns:
    wv[week] = pd.DataFrame(volumes_w[week].sort_values(ascending = False).head(10))
    wv[week].columns = ['Volume']

Making weekly volume DataFrames


In [17]:
volumes_w.columns

DatetimeIndex(['2016-01-10', '2016-01-17', '2016-01-24', '2016-01-31',
               '2016-02-07', '2016-02-14', '2016-02-21', '2016-02-28',
               '2016-03-06', '2016-03-13',
               ...
               '2018-05-06', '2018-05-13', '2018-05-20', '2018-05-27',
               '2018-06-03', '2018-06-10', '2018-06-17', '2018-06-24',
               '2018-07-01', '2018-07-08'],
              dtype='datetime64[ns]', length=131, freq='W-SUN')

In [ ]:
print('Coin Labels for 10 coins to track per week')
# Coin Labels for 10 coins to track per week
top_coins = list()
for week in wv:
    top_coins.append(wv[week].index) 

In [ ]:
print('Making weekly DataFrames: dv["date"]')
# Making weekly DataFrames
dv = {}
for (day, coin) in zip(volumes, top_coins):
    dv[day] = pd.DataFrame(volumes[day][coin].head(10))
    dv[day].columns = ['Volume']

In [ ]:
wv['Jan 10, 2016']

In [ ]:
print('Calculating Proportions')
print()
# Calculating Proportion of volume in total top 10
for day in volumes.columns:
    dv[day]['Proportion'] = (dv[day]['Volume'])/(dv[day]['Volume'].sum())
    
print(dv['Jan 01, 2016'])
print()
print('total:                  ', dv['Jan 01, 2016']['Proportion'].sum())

In [ ]:
print('Corresponding Close Price & pct_change columns')
print('Creating simples and log lists')

coin_labels = list()
yesterday_list = close_df.columns[1::]
simple_list = list()
log_list = list()

for day in dv:
    coin_labels.append(dv[day].index)      

# Adding close_price columns for top 10 coins
for (day, coin, prior) in zip(close_df.columns, coin_labels, yesterday_list):
    dv[day]['close_price'] = close_df[day][coin]
    dv[day]['prior_close'] = close_df[prior][coin]
    dv[day]['pct_change_close'] = close_pct[day][coin]
    dv[day]['simple_returns'] = dv[day]['Proportion']*dv[day]['pct_change_close']
    dv[day]['logs'] = np.log((dv[day]['close_price'])/(dv[day]['prior_close']))
    dv[day]['prop_logs'] = dv[day]['Proportion']*dv[day]['logs']
    simple_list.append(sum(dv[day]['simple_returns']))
    log_list.append(sum(dv[day]['prop_logs']))
    
    
print('Weekly DataFrames with: Volume, Proportion, Close_Price, Pct_Change')
print()
print('Jan 08, 2016')
# Setting NaN values of jan 1, 2016 to 0
dv['Jan 01, 2016']['pct_change_close'] = 0
dv['Jan 09, 2016']

In [ ]:
print('Daily Returns Dataframe')
Returns_D = pd.DataFrame(data = log_list, index = volumes_w.columns[1:], 
                       columns = ['logs'])

Returns_D.at['Jan 02, 2016','Simple_Returns'] = None
Returns_D['Cum_Simple_Returns'] = (1 + Returns_D['Simple_Returns']).cumprod()
Returns_D.at['Jan 02, 2016','Cum_Simple_Returns'] = 1
Returns_D['Cum_Log_Returns'] = Returns_D['Log_Returns'].cumsum()

Returns_D.head(5)

In [ ]:
volatility = np.std(Returns_D['Log_Returns'], axis = 0)*100
print('Volatility: ', volatility, '%')